# Veritas Fairness Assement - Life Insurance Underwriting Study (sample code)
This notebook includes samples of code used in the analysis conducted during the life insurance underwriting case study.

It is applicable to insurance underwriting datasets including a life insurance dataset available on
[kaggle](https://www.kaggle.com/c/prudential-life-insurance-assessment/data)

## License

Written by Sankarshan Mridha (Swiss Re) and Laura Alvarez (Accenture) as an extension to Phase 1 Credit Scoring Use Case code https://github.com/veritas-project/phase1/tree/main/credit_scoring 

Contact email: Veritas@mas.gov.sg


Copyright © 2021 Monetary Authority of Singapore

Licensed under the Apache License, Version 2.0 (the "License"); you may not use
this file except in compliance with the License. You may obtain a copy of the
License at http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed
under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR
CONDITIONS OF ANY KIND, either express or implied. See the License for the
specific language governing permissions and limitations under the Licens

## Imports

In [ ]:
# Core Packages
import os
import sys
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, brier_score_loss, precision_score,\
recall_score, balanced_accuracy_score
import joblib
import seaborn as sns

SEED = 123

In [ ]:
# Our code (autoreload)
%load_ext autoreload
%autoreload 2
sys.path.append("../utils")
import utility as utils

In [ ]:
# High-res plots
%config InlineBackend.figure_format = 'retina'

In [ ]:
import warnings
warnings.filterwarnings('ignore') 

## Load Data

Please modify the following cell to update dataset file path

In [ ]:
all_data = pd.read_csv('../dataset.csv')

## Feature Engineering

In [ ]:
all_data['BMI_Age'] = all_data['BMI'] * all_data['Ins_Age']

med_keyword_columns = all_data.columns[all_data.columns.str.startswith('Medical_Keyword_')]
all_data['Med_Keywords_Count'] = all_data[med_keyword_columns].sum(axis=1)

mapper = {
    'Id': 'Insured ID',
    'InsuredInfo_6': 'Gender',
    'InsuredInfo_1': 'Race',
    'InsuredInfo_4': 'Nationality',
    'Family_Hist_1': 'Marital Status',
    'InsuredInfo_3': 'Occupation Type',
    'Employment_Info_2': 'Occupation Industry',
    'Wt': 'Weight',
    'Ht': 'Height',
    'Medical_History_4': 'Smoker Status',
    'Ins_Age': 'Age at Policy Inception',
    'Insurance_History_3': 'No. of Life Policies',
    'Insurance_History_2': 'No. of Accident Policies',
    'Insurance_History_7': 'No. of CI Policies',
    'Product_Info_3': 'Duration in force for Medical Plan'
}

all_data.rename(mapper=mapper, axis=1, inplace=True)
# Drop columns we do not have confidence in mapping to
drop_columns = ('Medical', 'Family', 'Insurance', 'Product', 'Employment', 'Insurance', 'InsuredInfo')
mask = all_data.columns.str.startswith(drop_columns)
all_data = all_data.iloc[:,~mask]
all_data.head()

### Binary outcome labels

In [ ]:
# create labels
# 0: {1,2}
# 1: {7,8}
# -1: the rest
all_data['Risk'] = pd.cut(all_data.Response, bins=[0,2,6,8], labels=[0,-1,1])
all_data = all_data.astype({"Risk": int})
all_data.Risk.value_counts()

In [ ]:
# remove Response = -1
df = all_data.loc[all_data['Risk']!= -1].reset_index(drop=True)

####  Code corresponding to section 2.7.3 Step 3: Build and Validate in Veritas Document 4 FEAT Principles Assessment Case Studies

## Train/test split and preprocessing

In [ ]:
# prepare train & test datasets
columns_to_drop = ['Insured ID','Response','Risk', 'Nationality', 'Marital Status'] #droping race at pipeline point  
X = df.drop(columns=columns_to_drop)
X = X.astype({"Occupation Industry": object, "Occupation Type": object, "Smoker Status": object, "Gender": object})
y = df['Risk']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)
print(f"X_train.shape: {X_train.shape}, X_test.shape: {X_test.shape}")
print(f"y_train.shape: {y_train.shape}, y_test.shape: {y_test.shape}")

In [ ]:
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, selector(dtype_exclude=["object", "category"])),
    ('cat', categorical_transformer, selector(dtype_include=["object", "category"]))
], remainder='passthrough')

X_train_transformed = preprocessor.fit_transform(X_train.drop(columns='Race'))# dropping race prior to preprocessor
X_test_transformed = preprocessor.transform(X_test.drop(columns='Race'))# dropping race prior to preprocessor
print(f"X_train_transformed.shape: {X_train_transformed.shape}, X_test_transformed.shape: {X_test_transformed.shape}")

print(f"Class distribution: {np.unique(y_train, return_counts=True)}")

## Logistic Regression

In [ ]:
# logistic regression
model1 = LogisticRegression(max_iter=150, random_state=SEED)
model1.fit(X_train_transformed, y_train)

# predict probabilites
y_scores = model1.predict_proba(X_test_transformed)[:,1]

# compute AUC
print(roc_auc_score(y_test, y_scores))

In [ ]:
# compute ROC curve
fpr, tpr, th = roc_curve(y_test, y_scores)

In [ ]:
# find optimal cutoff by max balanced accuracy
ba = (tpr + (1 - fpr))/2
best_ba = np.max(ba)
best_th = th[np.argmax(ba)]
best_th

In [ ]:
# compute classification metrics by 0.5 cutoff
y_pred = np.where(y_scores > best_th, 1, 0)
print(classification_report(y_test, y_pred))

In [ ]:
# plot balanced accuracy and approval rate vs threshold
ba = 0.5*(tpr + 1 - fpr)
base_ar = np.mean(y_test.astype(int))
ar = base_ar*tpr + (1-base_ar)*fpr
plt.plot(th, ba, label='balanced accuracy')
plt.plot(th, ar, label='approval rate')
plt.plot()
plt.scatter(best_th, best_ba, c='r', marker='x', s=100, label='max bal acc')
plt.xlabel('Underwriting Threshold')
plt.title('Life Insurance Underwriting')
plt.xlim((y_pred.min(), y_pred.max()))
plt.legend(framealpha=0.3, facecolor='white', fontsize=12, loc='lower left')
plt.show()

## Test Performance
Here we quantify the model's performance.

####  Code corresponding to section 2.7.3 Step 3: Build and Validate in Veritas Document 4 FEAT Principles Assessment Case Studies

In [ ]:
# compute confusion matrix
cf_matrix = confusion_matrix(y_test, y_pred)

group_names = ['True Neg','False Pos','False Neg','True Pos']
group_counts = ['{0:0.0f}'.format(value) for value in cf_matrix.flatten()]

labels = [f"{v1}\n{v2}" for v1, v2 in zip(group_counts,group_names)]
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Blues')
plt.xlabel("Predicted Class")
plt.ylabel("True Label")
plt.show()

In [ ]:
# plot ROC curve
def plot_roc(model, X, y):
    figure = plt.figure(figsize=(5,5))
    plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
            label='Random Chance', alpha=.8)
    metrics.plot_roc_curve(model, X, y, name='model', alpha=0.3, lw=2, ax=plt.gca())
    plt.title('Receiver Operating Characteristic (ROC) Curve', fontsize=13)
    plt.xlabel('False Positive Rate', fontsize=12)
    plt.ylabel('True Positive Rate', fontsize=12)
    plt.show()

plot_roc(model1, X_test_transformed, y_test)

In [ ]:
# save model
joblib.dump(model1, 'model/model_baseline_lr.pkl')

In [ ]:
# Bootstrap uncertainty analysis

# Metrics based on predictions
prediction_metrics = {'True Positive Rate (i.e. sensitivity, or recall)': metrics.recall_score,
                      'True Negative Rate (i.e. specificity)': lambda x, y: metrics.recall_score(x, y, pos_label=0),
                      'Balanced Accuracy': metrics.balanced_accuracy_score,
                      'Positive Predictive Value (precision)': metrics.precision_score}

# Metrics based on probabilities
probability_metrics = {'Area Under ROC': metrics.roc_auc_score}

for name, metric_func in prediction_metrics.items():
    print(name, ":", utils.format_uncertainty(*utils.bootstrap_conf_int(y_test.values, y_pred, metric_func, k=25)))

for name, metric_func in probability_metrics.items():
    print(name, ":", utils.format_uncertainty(*utils.bootstrap_conf_int(y_test.values, y_scores, metric_func, k=25)))

In [ ]:
# Calibration curve
def plot_calibration(bin_true_prob, bin_pred_prob):
    plt.figure(figsize=(7, 7))
    ax1 = plt.subplot2grid((3, 1), (0, 0), rowspan=2)
    ax2 = plt.subplot2grid((3, 1), (2, 0))

    ax1.plot([0, 1], [0, 1], "k:", label="perfectly calibrated")
    ax1.plot(bin_pred_prob, bin_true_prob, "s-",
             label="model")

    ax2.hist([y_scores[y_test == 1], y_scores[y_test == 0]], label=["healthy", "risky"],
              histtype='bar', stacked=True)

    ax1.set_ylabel("Fraction of healthy applicants", fontsize=14)
    ax1.set_ylim([-0.05, 1.05])
    ax1.legend(loc="lower right", fontsize=12)
    ax1.set_title('Model Calibration (reliability curve)', fontsize=16)

    ax2.set_xlabel("Model Output Probabilities (binned)", fontsize=14)
    ax2.set_ylabel("Count", fontsize=14)
    ax2.legend(loc="upper left", ncol=2, fontsize=12)

    plt.tight_layout()
    plt.show()

bin_true_prob, bin_pred_prob = calibration_curve(y_test, y_scores, n_bins=10)
plot_calibration(bin_true_prob, bin_pred_prob)

In [ ]:
# run Isotonic calibration
clf_isotonic = CalibratedClassifierCV(model1, cv=3, method='isotonic')
clf_isotonic = clf_isotonic.fit(X_train_transformed, y_train)
model1_iso = clf_isotonic.predict_proba(X_test_transformed)[:, 1]
model1_score = brier_score_loss(y_test, y_scores)
clf_isotonic_score = brier_score_loss(y_test, model1_iso)
print(model1_score, clf_isotonic_score)